# AWS S3 Interaction with Boto3

This notebook demonstrates:
1. Setting up AWS credentials and Boto3 client
2. Creating an S3 bucket
3. Creating and uploading a test file
4. Listing bucket contents
5. Reading file contents from S3
6. Cleaning up resources (deleting files and buckets)

## Prerequisites
- AWS account with appropriate permissions
- AWS credentials configured (access key and secret key)
- Boto3 library installed

In [ ]:
# Install boto3 if needed (uncomment to install)
# !pip install boto3

In [ ]:
import boto3
import json
import os
from datetime import datetime
from botocore.exceptions import ClientError, NoCredentialsError
import uuid

print(f"Boto3 version: {boto3.__version__}")

## Step 1: Configure AWS Credentials

**Option 1: Use AWS credentials file** (Recommended)
- Configure via AWS CLI: `aws configure`
- Or manually create `~/.aws/credentials`

**Option 2: Set environment variables**
```bash
export AWS_ACCESS_KEY_ID='your-access-key'
export AWS_SECRET_ACCESS_KEY='your-secret-key'
export AWS_DEFAULT_REGION='us-east-1'
```

**Option 3: Specify credentials directly** (Not recommended for production)

In [ ]:
# Configuration
AWS_REGION = 'us-east-1'  # Change to your preferred region

# Option 1: Use default credentials (from ~/.aws/credentials or environment)
s3_client = boto3.client('s3', region_name=AWS_REGION)

# Option 2: Specify credentials directly (ONLY for testing - not recommended)
# s3_client = boto3.client(
#     's3',
#     region_name=AWS_REGION,
#     aws_access_key_id='YOUR_ACCESS_KEY',
#     aws_secret_access_key='YOUR_SECRET_KEY'
# )

# Test connection by listing existing buckets
try:
    response = s3_client.list_buckets()
    print("Successfully connected to AWS S3!")
    print(f"\nExisting buckets in your account:")
    if response['Buckets']:
        for bucket in response['Buckets']:
            print(f"  - {bucket['Name']} (Created: {bucket['CreationDate']})")
    else:
        print("  No existing buckets found.")
except NoCredentialsError:
    print("ERROR: AWS credentials not found!")
    print("Please configure your credentials using 'aws configure' or set environment variables.")
except ClientError as e:
    print(f"ERROR: {e}")

## Step 2: Create S3 Bucket

S3 bucket names must be globally unique across all AWS accounts.

In [ ]:
def create_s3_bucket(bucket_name, region=None):
    """
    Create an S3 bucket in a specified region.
    
    Args:
        bucket_name: Name of the bucket to create
        region: AWS region (if None, uses default region)
    
    Returns:
        True if bucket created successfully, False otherwise
    """
    try:
        if region is None or region == 'us-east-1':
            # us-east-1 doesn't need LocationConstraint
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            location = {'LocationConstraint': region}
            s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration=location
            )
        print(f"Successfully created bucket: {bucket_name}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'BucketAlreadyOwnedByYou':
            print(f"Bucket {bucket_name} already exists and is owned by you.")
            return True
        elif error_code == 'BucketAlreadyExists':
            print(f"ERROR: Bucket {bucket_name} already exists globally. Choose a different name.")
            return False
        else:
            print(f"ERROR creating bucket: {e}")
            return False

# Generate a unique bucket name
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
unique_id = str(uuid.uuid4())[:8]
BUCKET_NAME = f'test-bucket-{timestamp}-{unique_id}'.lower()

print(f"Creating bucket: {BUCKET_NAME}")
print(f"Region: {AWS_REGION}")
print("=" * 60)

bucket_created = create_s3_bucket(BUCKET_NAME, AWS_REGION)

if bucket_created:
    print(f"\nBucket '{BUCKET_NAME}' is ready to use!")

## Step 3: Create a Test File and Upload to S3

In [ ]:
def create_test_file(filename='test_file.txt', content=None):
    """
    Create a test file with sample content.
    
    Args:
        filename: Name of the file to create
        content: Content to write (if None, creates default content)
    
    Returns:
        Path to the created file
    """
    if content is None:
        content = f"""This is a test file for AWS S3 upload.
Created on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

Sample Data:
------------
Line 1: Hello from S3!
Line 2: This file demonstrates S3 upload functionality.
Line 3: Boto3 makes AWS integration easy.
Line 4: S3 is a scalable object storage service.
Line 5: End of test file.
"""
    
    with open(filename, 'w') as f:
        f.write(content)
    
    file_size = os.path.getsize(filename)
    print(f"Created test file: {filename} ({file_size} bytes)")
    return filename

def upload_file_to_s3(file_path, bucket, object_name=None):
    """
    Upload a file to an S3 bucket.
    
    Args:
        file_path: Path to file to upload
        bucket: Bucket to upload to
        object_name: S3 object name (if None, uses file_path)
    
    Returns:
        True if file was uploaded, False otherwise
    """
    if object_name is None:
        object_name = os.path.basename(file_path)
    
    try:
        s3_client.upload_file(file_path, bucket, object_name)
        print(f"Successfully uploaded {file_path} to {bucket}/{object_name}")
        return True
    except ClientError as e:
        print(f"ERROR uploading file: {e}")
        return False

# Create and upload test file
TEST_FILE = 'test_file.txt'
print("Creating test file...")
create_test_file(TEST_FILE)

print("\nUploading to S3...")
upload_success = upload_file_to_s3(TEST_FILE, BUCKET_NAME)

if upload_success:
    print(f"\nFile is now available at: s3://{BUCKET_NAME}/{TEST_FILE}")

## Step 4: Upload Additional Files (JSON Example)

In [ ]:
# Create and upload a JSON file
json_data = {
    'experiment': 'S3_test',
    'timestamp': datetime.now().isoformat(),
    'data': {
        'samples': ['sample1', 'sample2', 'sample3'],
        'results': [42, 37, 51],
        'passed': True
    },
    'metadata': {
        'version': '1.0',
        'author': 'Boto3 Script'
    }
}

JSON_FILE = 'test_data.json'
with open(JSON_FILE, 'w') as f:
    json.dump(json_data, f, indent=2)

print(f"Created JSON file: {JSON_FILE}")
print("\nJSON content:")
print(json.dumps(json_data, indent=2))

print("\nUploading JSON file to S3...")
upload_file_to_s3(JSON_FILE, BUCKET_NAME)

## Step 5: List Bucket Contents

In [ ]:
def list_bucket_contents(bucket_name):
    """
    List all objects in an S3 bucket.
    
    Args:
        bucket_name: Name of the bucket to list
    
    Returns:
        List of object keys
    """
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        
        if 'Contents' not in response:
            print(f"Bucket '{bucket_name}' is empty.")
            return []
        
        objects = response['Contents']
        print(f"Contents of bucket '{bucket_name}':")
        print("=" * 80)
        print(f"{'Object Key':<40} {'Size (bytes)':<15} {'Last Modified'}")
        print("=" * 80)
        
        object_keys = []
        for obj in objects:
            key = obj['Key']
            size = obj['Size']
            modified = obj['LastModified'].strftime('%Y-%m-%d %H:%M:%S')
            print(f"{key:<40} {size:<15} {modified}")
            object_keys.append(key)
        
        print("=" * 80)
        print(f"Total objects: {len(objects)}")
        print(f"Total size: {sum(obj['Size'] for obj in objects):,} bytes")
        
        return object_keys
        
    except ClientError as e:
        print(f"ERROR listing bucket contents: {e}")
        return []

# List all objects in the bucket
object_list = list_bucket_contents(BUCKET_NAME)

## Step 6: Read and Display File Contents from S3

In [ ]:
def read_s3_object(bucket_name, object_key):
    """
    Read and return the contents of an S3 object.
    
    Args:
        bucket_name: Name of the bucket
        object_key: Key of the object to read
    
    Returns:
        Content of the object as string
    """
    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=object_key)
        content = response['Body'].read().decode('utf-8')
        return content
    except ClientError as e:
        print(f"ERROR reading object: {e}")
        return None

# Read and display the text file
print(f"Reading file: {TEST_FILE}")
print("=" * 60)
text_content = read_s3_object(BUCKET_NAME, TEST_FILE)
if text_content:
    print(text_content)
    print("=" * 60)

# Read and display the JSON file
print(f"\nReading file: {JSON_FILE}")
print("=" * 60)
json_content = read_s3_object(BUCKET_NAME, JSON_FILE)
if json_content:
    json_parsed = json.loads(json_content)
    print(json.dumps(json_parsed, indent=2))
    print("=" * 60)

## Step 7: Get Object Metadata

In [ ]:
def get_object_metadata(bucket_name, object_key):
    """
    Get metadata for an S3 object.
    
    Args:
        bucket_name: Name of the bucket
        object_key: Key of the object
    """
    try:
        response = s3_client.head_object(Bucket=bucket_name, Key=object_key)
        
        print(f"Metadata for: s3://{bucket_name}/{object_key}")
        print("=" * 60)
        print(f"Content-Type: {response.get('ContentType', 'N/A')}")
        print(f"Content-Length: {response.get('ContentLength', 0):,} bytes")
        print(f"Last-Modified: {response.get('LastModified', 'N/A')}")
        print(f"ETag: {response.get('ETag', 'N/A')}")
        print(f"Storage-Class: {response.get('StorageClass', 'STANDARD')}")
        
        if 'Metadata' in response and response['Metadata']:
            print("\nCustom Metadata:")
            for key, value in response['Metadata'].items():
                print(f"  {key}: {value}")
        print("=" * 60)
        
    except ClientError as e:
        print(f"ERROR getting metadata: {e}")

# Get metadata for uploaded files
for obj_key in object_list:
    get_object_metadata(BUCKET_NAME, obj_key)
    print()

## Step 8: Download Files from S3

In [ ]:
def download_file_from_s3(bucket_name, object_key, local_filename=None):
    """
    Download a file from S3 to local filesystem.
    
    Args:
        bucket_name: Name of the bucket
        object_key: Key of the object to download
        local_filename: Local file path (if None, uses object_key)
    
    Returns:
        True if successful, False otherwise
    """
    if local_filename is None:
        local_filename = f"downloaded_{object_key}"
    
    try:
        s3_client.download_file(bucket_name, object_key, local_filename)
        file_size = os.path.getsize(local_filename)
        print(f"Successfully downloaded {object_key} to {local_filename} ({file_size} bytes)")
        return True
    except ClientError as e:
        print(f"ERROR downloading file: {e}")
        return False

# Example: Download the JSON file with a different name
print("Downloading file from S3...")
download_file_from_s3(BUCKET_NAME, JSON_FILE, 'downloaded_test_data.json')

## Step 9: Cleanup - Delete Files and Bucket

**WARNING:** The following cells will permanently delete files and the bucket.

Run these cells only when you're ready to clean up the resources.

In [ ]:
def delete_object_from_s3(bucket_name, object_key):
    """
    Delete an object from S3.
    
    Args:
        bucket_name: Name of the bucket
        object_key: Key of the object to delete
    
    Returns:
        True if successful, False otherwise
    """
    try:
        s3_client.delete_object(Bucket=bucket_name, Key=object_key)
        print(f"Deleted: s3://{bucket_name}/{object_key}")
        return True
    except ClientError as e:
        print(f"ERROR deleting object: {e}")
        return False

def delete_all_objects_in_bucket(bucket_name):
    """
    Delete all objects in a bucket.
    
    Args:
        bucket_name: Name of the bucket
    """
    try:
        response = s3_client.list_objects_v2(Bucket=bucket_name)
        
        if 'Contents' not in response:
            print(f"Bucket '{bucket_name}' is already empty.")
            return
        
        print(f"Deleting all objects from bucket '{bucket_name}'...")
        for obj in response['Contents']:
            delete_object_from_s3(bucket_name, obj['Key'])
        
        print(f"\nAll objects deleted from '{bucket_name}'.")
        
    except ClientError as e:
        print(f"ERROR deleting objects: {e}")

# Option 1: Delete specific file
print("Option 1: Delete specific files")
print("=" * 60)
print("\nTo delete a specific file, uncomment and run:")
print(f"# delete_object_from_s3('{BUCKET_NAME}', '{TEST_FILE}')")
print(f"# delete_object_from_s3('{BUCKET_NAME}', '{JSON_FILE}')")

# Uncomment to delete specific files:
# delete_object_from_s3(BUCKET_NAME, TEST_FILE)
# delete_object_from_s3(BUCKET_NAME, JSON_FILE)

In [ ]:
# Option 2: Delete all objects in the bucket
print("Option 2: Delete ALL objects in bucket")
print("=" * 60)
print(f"Bucket: {BUCKET_NAME}")
print("\nWARNING: This will delete ALL files in the bucket!")
print("\nTo delete all objects, uncomment and run:")
print(f"# delete_all_objects_in_bucket('{BUCKET_NAME}')")

# Uncomment to delete all objects:
# delete_all_objects_in_bucket(BUCKET_NAME)

In [ ]:
def delete_bucket(bucket_name):
    """
    Delete an S3 bucket (must be empty).
    
    Args:
        bucket_name: Name of the bucket to delete
    
    Returns:
        True if successful, False otherwise
    """
    try:
        s3_client.delete_bucket(Bucket=bucket_name)
        print(f"Successfully deleted bucket: {bucket_name}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'BucketNotEmpty':
            print(f"ERROR: Bucket '{bucket_name}' is not empty.")
            print("Delete all objects first before deleting the bucket.")
        else:
            print(f"ERROR deleting bucket: {e}")
        return False

# Delete the bucket
print("Option 3: Delete the S3 bucket")
print("=" * 60)
print(f"Bucket: {BUCKET_NAME}")
print("\nWARNING: The bucket must be empty before deletion!")
print("\nTo delete the bucket, uncomment and run:")
print(f"# delete_bucket('{BUCKET_NAME}')")

# Uncomment to delete the bucket:
# delete_bucket(BUCKET_NAME)

In [ ]:
# Complete cleanup function - deletes everything
def complete_cleanup(bucket_name):
    """
    Complete cleanup: delete all objects and the bucket.
    
    Args:
        bucket_name: Name of the bucket to clean up
    """
    print(f"Starting complete cleanup for bucket: {bucket_name}")
    print("=" * 60)
    
    # Step 1: Delete all objects
    print("\nStep 1: Deleting all objects...")
    delete_all_objects_in_bucket(bucket_name)
    
    # Step 2: Delete the bucket
    print("\nStep 2: Deleting bucket...")
    if delete_bucket(bucket_name):
        print("\nCleanup complete! All resources have been removed.")
    else:
        print("\nCleanup incomplete. Please check errors above.")

print("Complete Cleanup Option")
print("=" * 60)
print(f"Bucket: {BUCKET_NAME}")
print("\nWARNING: This will delete ALL files and the bucket!")
print("\nTo perform complete cleanup, uncomment and run:")
print(f"# complete_cleanup('{BUCKET_NAME}')")

# Uncomment to perform complete cleanup:
# complete_cleanup(BUCKET_NAME)

## Step 10: Clean Up Local Files (Optional)

In [ ]:
# Clean up local test files
def cleanup_local_files():
    """
    Remove local test files created during this session.
    """
    local_files = [TEST_FILE, JSON_FILE, 'downloaded_test_data.json']
    
    print("Cleaning up local files...")
    for filename in local_files:
        if os.path.exists(filename):
            os.remove(filename)
            print(f"Removed: {filename}")
        else:
            print(f"Not found (skipping): {filename}")
    
    print("\nLocal cleanup complete!")

print("Local File Cleanup")
print("=" * 60)
print("To remove local test files, uncomment and run:")
print("# cleanup_local_files()")

# Uncomment to clean up local files:
# cleanup_local_files()

## Summary

This notebook demonstrated:

1. **AWS S3 Connection** - Connected to AWS using Boto3 with proper credential handling
2. **Bucket Creation** - Created a uniquely named S3 bucket
3. **File Upload** - Created and uploaded text and JSON files to S3
4. **Bucket Listing** - Listed all objects in the bucket with metadata
5. **File Reading** - Read and displayed contents of files stored in S3
6. **File Download** - Downloaded files from S3 to local filesystem
7. **Cleanup Options** - Provided multiple options for resource cleanup:
   - Delete specific files
   - Delete all files in bucket
   - Delete the bucket
   - Complete cleanup (all of the above)
   - Clean up local test files

### Key Takeaways:

- S3 bucket names must be globally unique
- Always handle AWS credentials securely
- Buckets must be empty before deletion
- Use proper error handling for AWS operations
- Consider costs when storing data in S3

### Next Steps:

- Explore S3 versioning and lifecycle policies
- Implement server-side encryption
- Set up bucket policies and access controls
- Use S3 Select for querying data
- Integrate with other AWS services (Lambda, EC2, etc.)